In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from tensorflow import keras

# Modelling Regression Problems
In a regression problem the model needs to predict a continuous real number. For example, predict the price of a house based on its area, number of rooms and zip code.

## Linear Regression
In **linear regression** a linear relationship is said to exist between the features and the label. For example, if the house floor area is feature $x_1$ and number of rooms is $x_2$ then we can say that the price $y$ of the house relates to them as follows.

$$y = w_1x_1 + w_2x_2 + b$$

<img src="https://bibhas2.github.io/gradient/n-feature.svg"  height="170"/>

In a linear equation we usually call $w_1$ and $w_2$ coefficients or slopes. And $b$ is called the intercept. In machine learning we call $w_i$ weights and $b$ bias. Collectively they are both just called weights.

## What Does the Machine Learn?
An algorithm (or model) will look at the previously observed features and labels and try to learn the weights that best describe the relationship between them. 

During the learning process the feature and label values ($y$, $x_1$ and $x_2$) are known real numbers. The weights ($w_1$, $w_2$ and $b$) are variables. During learning we keep adjusting the weights until we get a most optimal fit.

Once learning is over the job of a model is to predict the label for any feature values. At this point the weights are fixed but the features can be any value. 

## Loss Function
Let us say that the real price of a house is $y$ and our model predicts it to be $\hat{y}$. Obviously there will be some discrepency between the two. A model's job will be to minimize this error for all known data points.

Our model needs to define a function that will measure the error in its predictions. This is called an **error function** or **loss function**. We will use the latter term in this course.

Let's first measure the error for a single house. Loss is often shown using the variable name $J$. We can write it like this.

$$loss = J = (y - \hat{y})$$

But that number can be positive or negative. For an absolute measure of loss we can write it like this.

$$J = |(y - \hat{y})|$$

It is a good measure of error. But an absolute function is not continuous and doesn't always produce good result. If we take a square of the error we get a smooth positive value.

$$J = (y - \hat{y})^2$$

This is a very popular measure of error and works in very many circumstances.

Now let's sum up the loss for all houses.

$$J = \sum\limits_{i=1}^{m} (\hat{y_i} - y_i)^2$$

Where $m$ is the number of houses, also called the sample size.

But the number above will depend on the number of houses. For a true measure of error we should take an average. 

$$J = \frac{1}{m}\sum\limits_{i=1}^{m} (\hat{y_i} - y_i)^2$$

For reasons we can not get into here it simplifies matters if we further divide the error by 2. Hence our final loss function will be:

$$J = \frac{1}{2m}\sum\limits_{i=1}^{m} (\hat{y_i} - y_i)^2$$

This loss function is known as the **Root Mean Square Error** or RMSE for short.

## Gradient Descent
During training our model continuously adjusts the weights so as to minimize the error. Gradient Descent is one of the simplest minimization algorithms. 

We start the training process with some random values for the weights. We then adjust the weights as follows.

$$w_1 = w_1 - \alpha \frac{\partial J}{w_1}$$

$$w_2 = w_2 - \alpha \frac{\partial J}{w_2}$$

$$b = b - \alpha \frac{\partial J}{b}$$

Here $\alpha$ is called the learning rate. It determines the extent to which we correct the weights.

After the weights are adjusted we repeat the correction process. We keep doing it until we see no appreciable change in the error. That will signify the lowest error we can achieve. We stop the training process at that point.

## Linear Regression Using Gardient Descent
We will now solve a simple regression problem using gradient descent. The algorithms are so simple that we can do this using the low level Tensrflow API. For more real life problems we will need to use high level APIs that we will learn later.

We observed the following housing market data points in Raleigh, NC area in 2020.

Area 1K sqft (x1) | Rooms (x2) | Price 1K (y)
--- | --- | ---
3.5 | 8 | 500
2.0 | 6 | 345
2.3 | 6 | 349
3.1 | 7 | 464
3.2 | 9 | 604
2.0 | 5 | 300
3.7 | 8 | 580

We need to train a model that will learn the weights so that we can predict the house prices in future.

### The Data Points
Below we capture the data points using numpy arrays.

In [ ]:
#Features: [area, rooms]
train_x = np.array([
  [3.5, 8],
  [2.0, 6],
  [2.3, 6],
  [3.1, 7],
  [3.2, 9],
  [2.0, 5],
  [3.7, 8]
])

#Labels: [price]
train_y = np.array([
  [500.0],
  [345.0],
  [349.0],
  [464.0],
  [604.0],
  [300.0],
  [580.0]
])

### Visualize the Data Points
Visualizing helps you to get some understanding of the data you are dealing with. This step is not exactly necessary but recommended.

In [ ]:
area = train_x[:,0] #Take a slice of the first column
plt.scatter(area, train_y)
plt.xlabel('Area')
plt.ylabel('Price')
plt.show()

rooms = train_x[:,1] #Take a slice of the 2nd column
plt.scatter(rooms, train_y)
plt.xlabel('Rooms')
plt.ylabel('Price')
plt.show()

Unsurprisingly prices go up with larger area and more rooms. The relationship looks fairly linear for both features. We can expect to get good results with linear regression.

### The Prediction Function
Earlier we have shown the linear relationship between the features and label as follows.

$$y = w_1x_1 + w_2x_2 + b$$

We can write this in vector form.

$$
y = \begin{bmatrix} 
x_1 & x_2
\end{bmatrix}
\times
\begin{bmatrix} 
w_1 \\
w_2
\end{bmatrix}
+ b
$$

In fact, with this vector form we can compute the label for many houses all at once. For example, if there are $m$ houses in the data sample, we can predict their prices like this.

$$
\begin{bmatrix} 
y1 \\
y2 \\
... \\
y_m
\end{bmatrix}
 = \begin{bmatrix} 
x_{11} & x_{12} \\
x_{21} & x_{22} \\
... & ...\\
x_{m1} & x_{m2}
\end{bmatrix}
\times
\begin{bmatrix} 
w_1 \\
w_2
\end{bmatrix}
+ b
$$

We can write this using Tensorflow API.

In [ ]:
def predict(x, W, b):
  return tf.add(tf.matmul(x, W), b)

### The Loss Function
The RMSE function is modeled like this using Tensorflow API.

In [ ]:
#y - Actually observed price
#y_ - Predicted price
def loss(y_, y):
  return tf.reduce_mean(tf.square(y_ - y))

### The Training Function
During training we calculate the partial derivative of the loss function and apply correction to the weights. In Tensorflow we can compute partial derivatives using ``tf.GradientTape()``. 

In [ ]:
def train(train_x, train_y, W, b):
  learning_rate = 0.01

  for epoch in range(10000):
    #Take a partial derivative of loss function
    with tf.GradientTape() as t:
      #Calculate loss
      J = loss(predict(train_x, W, b), train_y)
      #Obtain partial derivatives
      [dJ_dW, dJ_db] = t.gradient(J, [W, b])

    #Update W and b by subtracting the corrections
    W.assign_sub(learning_rate * dJ_dW)
    b.assign_sub(learning_rate * dJ_db)

    #Show training progress
    if epoch % 500 == 0:
      print(f"Epoch: {epoch} Loss: {J}")

Learning rate is a hyperparameter. You can experiment with different values. In this case ``0.01`` works well for us.

### Run Training and Prediction

In [ ]:
def housing_test():
  # Create the 2 weights initialized with random values.
  W = tf.Variable(tf.random.truncated_normal([2, 1], 
                    stddev=0.1, dtype=tf.dtypes.double))
  #Create the bias
  b = tf.Variable(0.0, dtype=tf.dtypes.double)

  #Run training
  train(train_x, train_y, W, b)

  #Run prediction
  predictions = predict(train_x, W, b)

  #Compare the predicted prices with the actual prices
  for y_, y in zip(predictions, train_y):
    print('Predicted:', y_.numpy(), 'Actual:', y)
    
housing_test()

As you can see the loss starts to go down as training progresses. After about epoch 6000 the loss stabilizes.

Our predictions are not bad considering we have trained with only a very small dataset.

### Quiz

Assuming there are $m$ samples in the training dataset:

1. When creating the ``W`` tensor what shape did we request it to be? Does that match the math equation we have seen earlier?
2. What is the shape of ``train_x``?
3. What is the shape of ``train_y``?
4. What are the dimensions of the ``predictions`` tensor?
5. The ``predictions`` and ``train_y`` tensors should have the same shape. True or False?
6. In the ``predict()`` function what will happen if we changed the order of input to ``tf.matmul()`` and make it ``tf.matmul(W, x)``?

Answers:

1. (2, 1). Yes, the equation shows a $2\times1$ matrix for the weights.
2. (m, 2).
3. (m, 1).
4. (m, 1).
5. True. They both contain price values.
6. That will not be a valid matrix multiplication and you will get a runtime error. Refer to the math equations to see the valid order of multiplication.

## Regression Using Neural Network
Neural Network builds on everything we have learned from linear regression. Here we arrange multiple nodes (or neurons) in layers. Each node receives a set of inputs. Each node has its own set of weights and a bias for the inputs. Output is first calculated using the same linear relationship. This output is then modified using an activation function such as ``sigmoid`` and ``relu``.

For each node the computation goes as follows.

$$y = w_1x_1 + w_2x_2 + ... + w_nx_n + b$$
$$y = activation(y)$$

### Activation Functions
Application of an activation function adds magic to an otherwise linear computation. This lets a neural network simulate any function in the universe - linear or non-linear.

#### Rectified Linear Unit (RELU)
This is a popular activation function. It is defined as a piecewise function.

$$
relu(x) = x;  x \gt 0
$$

$$
relu(x) = 0; x \leq 0
$$

We can see it in action.

In [ ]:
def relu(x):
    return x * (x > 0)
  
x = np.arange(-50, 50)

plt.plot(x, relu(x))

plt.show()

#### Hyperbolic Tangent (tanh)

The hyperbolic tangent function is also frequently used.

In [ ]:
def tanh(x):
  return np.tanh(x)

plt.plot(x, tanh(x))

plt.show()

Activation function can be seen as a hyperparameter. You need to experiment with different functions and choose the one that gives you the best result. At this point we do not have a clear understanding of exactly how some functions work better than others. Reserchers are coming up with new functions all the time. In this course we will mostly stick to RELU.

## Ames, Iowa House Price Prediction Using Neural Network
Citation:
- [Author](http://jse.amstat.org/v19n3/decock.pdf).
- [Original Dataset](http://www.amstat.org/publications/jse/v19n3/decock/AmesHousing.xls).

We will now use linear regression to predict house prices. Open ``ames_train.csv`` in Excel and follow the description of the data below. 

The file has many columns. But we will only use these:

- **SalePrice** - The sale price. This is the label and what we need to predict.
- **Lot Area** - Area of the lot in sqft.
- **Neighborhood** - The name of the neighborhood. Such as "CollgCr" for Collage Circle. This is a categorical column. We need to one-hot encode this to represent this as numbers.
- **Bldg Type** - The type of property. For example, "1Fam" for a single family house. This is a categorical column and needs to be one-hot encoded.
- **Full Bath** - Number of full bathrooms.
- **Half Bath** - Number of small bathrooms.
- **Kitchen AbvGr** - Number of kitchens above grade.
- **TotRms AbvGrd** - Number of rooms above grade excluding kitchen.
- **Garage Area** - Garage area. 0 if no garage.
- **Overall Qual** - Overall quality score from 1-10.
- **Overall Cond** - Overall condition score from 1-10. 
- **Year Built** - Year built.
- **Total Bsmt SF** - Basement area.
- **Central Air** - 'Y' if the house has central air. 'N' otherwise. This is a categorical feature and needs to be one-hot encoded.

### Upload the Data
Run the code below and upload ``AmesHousing.xls``.

In [ ]:
#Upload AmesHousing.xls
files.upload()

### Analyze the Data
Run this code to see the distribution of prices.

In [ ]:
prices = pd.read_excel("AmesHousing.xls", usecols=["SalePrice"])
plt.hist(prices.values, bins=20)
plt.xlabel('Prices')
plt.show()

There are a few issues with this data.

- Prices are heavily skewed in the \$100K to \$200K range. This can be fixed by taking a log of the prices.
- There are a few samples above \$600K and less than \$20K. There are so few of them that we should probably discard them as outliers.

Below we cleanup the prices and plot their distribution. Now they look evenly distributed.

In [ ]:
clean_prices = prices[(prices["SalePrice"] > 50000) & (prices["SalePrice"] < 600000)]
clean_prices = np.log(clean_prices)

plt.hist(clean_prices.values, bins=20)
plt.xlabel('Log of prices')
plt.show()

### Load and Cleanup Data
A good amount of effort in machine learning goes into cleaning up and preparing data. We will use the Pandas package to load the data. We will process the data as follows:

- One-hot encode categorical features.
- Clean up the prices as discussed above.
- Separate out the **SalePrice** column as label from the rest of the feature columns. 
- Split the data in training and testing sets.

In [ ]:
def load_ames_data(file_name):
  columns_used = ["SalePrice", "Lot Area", "Neighborhood", "Bldg Type", "Full Bath", "Half Bath", "Kitchen AbvGr", "TotRms AbvGrd", "Garage Area", "Overall Qual", "Overall Cond", "Year Built", "Total Bsmt SF", "Central Air"]
  source_data = pd.read_excel(file_name, usecols=columns_used)

  source_data = source_data.dropna()

  #One hot encode categorical data
  category_features = ['Neighborhood', 'Bldg Type', 'Central Air']
  source_data = pd.get_dummies(source_data, columns=category_features)

  #Filter out outlier prices
  source_data = source_data[(source_data["SalePrice"] > 20000) & (source_data["SalePrice"] < 600000)]

  #Do a log of price
  source_data["SalePrice"] = np.log(source_data["SalePrice"])

  #Split 90% of the data for training and 10% for test
  train = source_data.sample(frac=0.9, random_state=200)
  test = source_data.drop(train.index)

  #Extract the price column
  train_y = train["SalePrice"]
  test_y = test["SalePrice"]
  #Drop the price column
  train_x = train.drop("SalePrice", axis=1)
  test_x = test.drop("SalePrice", axis=1)

  #Return training and test features and labels
  return train_x, train_y, test_x, test_y

train_x, train_y, test_x, test_y = load_ames_data("AmesHousing.xls")
num_features=len(train_x.columns)

train_x.head(5)

### Model the Network
- Use RMSE error function.
- Use Adam optimizer.

In [ ]:
def create_ames_model(num_features):
  model = keras.Sequential()
  model.add(keras.layers.Dense(120, input_dim=num_features, activation='relu'))
  model.add(keras.layers.Dense(75, activation='relu'))
  model.add(keras.layers.Dense(50, activation='relu'))
  model.add(keras.layers.Dense(45, activation='relu'))
  model.add(keras.layers.Dense(25, activation='relu'))

  model.add(keras.layers.Dense(1)) #Readout

  model.compile(loss='mean_squared_error',
                  optimizer="adam")
  
  return model

def train_ames_model(model, features, label):
  model.fit(features.values, label.values, epochs=1000, batch_size=128, verbose=True)

model = create_ames_model(num_features)

train_ames_model(model, train_x, train_y)

### Test the Model

In [ ]:
def plot_prediction(prediction, reality):
  plt.plot(prediction, reality, 'ro')
  plt.xlabel('Predictions')
  plt.ylabel('Reality')
  plt.plot([reality.min(), reality.max()], [reality.min(), reality.max()], 'k--', lw=4)
  plt.show()

def test_ames_model(model, features, label):
  result = model.evaluate(x=features.values, y=label.values)
  #Print the loss
  print("Loss in testing:", result)

  predict_y = model.predict(features.values)

  #Convert prices back to actual values
  predict_y = np.exp(predict_y)
  label = np.exp(label.values)

  plot_prediction(predict_y, label)

test_ames_model(model, test_x, test_y)

### Quiz
1. Why did we take a log of the prices?
2. In the ``load_data()`` function what does ``source_data.dropna()`` do?
3. Once prediction is made what did we have to do to get the actual predicted price values?
4. In the ``create_model()`` function why does the final layer output a single value? As in: ``keras.layers.Dense(1)``.
5. What optimizer was used in this model?

Answers:

1. The prices are heavily skewed in a certain range. Taking a log evenly distributes the prices which makes it easier to minimize error during training.
2. It will remove any rows that have missing values.
3. We had to do an exponential ``np.exp()`` of the prediected values to get the actual prices.
4. Our model needs to predict a single value (per house) - its price. This is why the final layer outputs only one value.
5. We used the Adam optimizer.

## AirBnB Rent Prediction
We will now use neural network to solve another real life regression problem. We have the AirBnB property rental price data for the Boston, MA area. There are dozens of features that prices depend on. We will train a model to learn from the data and start to do fairly accurate prediction of prices.

## Understand the Data
AirBnb data for Boston is available in ``boston_listings.csv``. This was downloaded from: http://insideairbnb.com/get-the-data.html. 

Although there are many features in this file we decided to only use these features. Open the file in Excel and briefly check each feature.

- **property_type** - Categorical data. Example "Apartment".
- **room_type** - Categorical data
- **bathrooms** - Numerical but sometimes missing
- **bedrooms** - Numerical but sometimes missing
- **beds** - Numerical but sometimes missing
- **bed_type** - Categorical data
- **accommodates** - Numerical
- **host_total_listings_count** - Numerical
- **number_of_reviews** - Numerical
- **review_scores_value** - Numerical but sometimes missing
- **neighbourhood_cleansed** - Categorical data. Example: "Jamaica Plain" and "West Roxbury".
- **cleaning_fee** - Given as a string like ``$12.34``. Need conversion to numeric 
- **minimum_nights** - Numerical
- **security_deposit** - Given as a string like ``$12.34``. Need conversion to numeric 
- **host_is_superhost** - Categorical data. t for true and f for false
- **instant_bookable** - Categorical data. t for true and f for false
- **price** - This is the label column. Given as a string like ``$12.34``. Need conversion to numeric.

## Upload Data
Run the code below and upload **boston_listings.csv** from your machine.

In [ ]:
#Upload boston_listings.csv file
files.upload()

## Prepare Data
- Extract only the columns used from **boston_listings.csv**.
- Convert prices from text "$23.99" to number 23.99.
- Fill in missing values with median values or reasonable defaults
- Convert categorical data such as property type to one hot encoded values.
- Split data into train and test and save in **boston_train.csv** and **boston_test.csv** respectively.

With this approach we clean up the data only once. Training and testing work on the already prepared data.

In [ ]:
#Prepare data. This function will:
def prepare_boston_data():
  used_features = [
    'property_type', 'room_type', 'bathrooms', 'bedrooms', 'beds', 'bed_type',
    'accommodates', 'host_total_listings_count', 'number_of_reviews',
    'review_scores_value', 'neighbourhood_cleansed', 'cleaning_fee',
    'minimum_nights', 'security_deposit', 'host_is_superhost',
    'instant_bookable', 'price'
  ]

  # Load only the columns we need
  source_data = pd.read_csv('boston_listings.csv', usecols=used_features)

  # Convert string to numerical data
  for feature in ["cleaning_fee", "security_deposit", "price"]:
      source_data[feature] = source_data[feature].map(
          lambda x: x.replace("$", '').replace(",", ''), na_action='ignore')
      source_data[feature] = source_data[feature].astype(float)
      #Fill in missing values
      source_data[feature].fillna(source_data[feature].median(), inplace=True)

  # Fill missing values with median
  for feature in ["bathrooms", "bedrooms", "beds", "review_scores_value"]:
      source_data[feature].fillna(source_data[feature].median(), inplace=True)

  # Fill in missing data with reasonable defualt.
  # If this is not possible we need to exclude such bad data
  source_data['property_type'].fillna('Apartment', inplace=True)
  source_data['host_is_superhost'].fillna('f', inplace=True)
  source_data['host_total_listings_count'].fillna(1, inplace=True)

  #Filter out too expensive and too cheap properties
  source_data = source_data[(source_data["price"] > 20)
                            & (source_data["price"] < 1000)]

  # Spread out price by taking a log. Otherwise prices are very close to each other
  # which makes prediction less accurate
  source_data["price"] = np.log(source_data["price"])

  #One hot encode categorical data
  category_features = ['property_type', 'room_type', 'bed_type', 'neighbourhood_cleansed', 'host_is_superhost','instant_bookable']
  source_data = pd.get_dummies(source_data, columns=category_features, drop_first=True)

  # Split source as 80% training and 20% test data
  train = source_data.sample(frac=0.8, random_state=200)
  test = source_data.drop(train.index)


  # Save split data in files. No need to save the index column.
  train.to_csv("boston_train.csv", header=True, index=False)
  test.to_csv("boston_test.csv", header=True, index=False)

prepare_boston_data()

Write a routine to load the prepared data.

In [ ]:
#Return data from file as a [feature, label] tuple
#feature - is a pandas.DataFrame
#label - is a pandas.Series
def load_boston_data(file_name):
  source_data = pd.read_csv(file_name)

  #Extract the price column
  price_column = source_data["price"]
  #Drop the price column
  source_data.drop("price", axis=1)

  return source_data, price_column

### Build the Model
The neural network will have this architecture:

- Each input will have all feature columns.
- Hidden layer #1 will have 25 neurons
- Hidden layer #2 will have 15 neurons
- Hidden layer #3 will have 7 neurons
- The final readout layer will have 1 output for the prices.
- Use RMSE error function.
- Use Adam optimizer.

Exactly how many layers and how many neurons in a layer you need is something you need to experiment with.


In [ ]:
def create_boston_model(num_features):
  model = keras.Sequential()
  model.add(keras.layers.Dense(25, input_dim=num_features, activation='relu'))
  model.add(keras.layers.Dense(15, activation='relu'))
  model.add(keras.layers.Dense(7, activation='relu'))
  model.add(keras.layers.Dense(1)) #Readout

  model.compile(loss='mean_squared_error',
                  optimizer="adam")
  
  return model

### Train the Model
As before, we will call ``model.fit()`` to start training the model. There is one key difference. Now we will save the model and all of its weights in the ``boston-model/`` folder. This folder becomes the main deliverable from the training process. During the prediction phase we will load the model and the weights from this folder.

There is very good interoperability between languages for the saved data format. For example, you can load the model in Java and with some conversion from Swift.

In [ ]:
def train_boston_model():
  #Start training
  features, label = load_boston_data("boston_train.csv")
  model = create_boston_model(len(features.columns))
  model.fit(features.values, label.values, epochs=1000, batch_size=128, verbose=True)

  #Save the model and all weights
  model.save('boston-model')

train_boston_model()

Evaluate and test the model. You should see very good results (a loss of about 0.08). Also note how we are creating the model by loading it from the ``boston-model/`` folder.

In [ ]:
def test_boston_model():
  features, label = load_boston_data("boston_test.csv")

  #Load the model and all weights
  model = keras.models.load_model('boston-model')
  
  #Run evaluation
  result = model.evaluate(x=features.values, y=label.values)
  #Print the loss
  print(result)

  predict_y = model.predict(features.values)
  for y, y_ in zip(label.values, predict_y):
    # Print the actual price from log values
    print(np.exp(y), np.exp(y_))

  plot_prediction(predict_y, label)

test_boston_model()

## Lung Capacity Prediction
Forced Expiratory Volume (FEV) is the amount of air you can force from your lungs in one second. It is a good measure lung capacity and is used to evaluate chronic obstructive pulmonary disease.

In this challenge you will build a neural network to predict FEV among teenagers based on the following features:

- age
- height
- sex. Encoded as 0 or 1.
- smoke. 1 if smokes or 1.

The ``sex`` and ``smoke`` features will be normally categorical. But the source data already encodes them as 1 or 0. So no further processing is needed.

This workshop is self paced. Many hints are given to you as aids. The solution is available below in case you need it.

### Upload File
Upload the source data file **lung-capacity.txt**.

In [ ]:
#Upload lung-capacity.txt file
files.upload()

### Inspect the Data
Run the code below to have a look at the column names.

In [ ]:
data = pd.read_csv('lung-capacity.txt', delim_whitespace=True)
data.head(5)

### Load Data
Carry out these tasks from the ``load_fev_data()`` function.

- Load the space separated data file.
- Do a 90-10 split of training and test data
- Extract the ``FEV`` column to create the label.
- Drop the ``FEV`` column from the source to create the feature
- Return the training and test feature and label.

In [ ]:
def load_fev_data():
  #Load space separated data
  fev_data = pd.read_csv('lung-capacity.txt', delim_whitespace=True)

  #Split 90% of the data for training and 10% for test
  train = 
  test = 

  #Extract the FEV column
  train_y = 
  test_y = 

  #Drop the FEV column
  train_x = 
  test_x = 

  #Return data
  return train_x, train_y, test_x, test_y

### Create the Model
Create a model with this architecture:

- Hidden layer #1 has 25 neurons (nodes). Activation: relu.
- Hidden layer #2 has 10 neurons (nodes). Activation: relu.
- Hidden layer #3 has 5 neurons (nodes). Activation: relu.
- Final readout layer outputs one value.

Compile the model with RMSE error function and Adam optimizer.

In [ ]:
def create_fev_model(num_features):
  model = keras.Sequential()

  #Hidden layer #1

  #Hidden layer #2

  #Hidden layer #3

  #Readout layer

  #Compile model
  
  return model

### Train the Model

In [ ]:
def train_fev_model():
  #Load training data
  train_x, train_y, _, _ = load_fev_data()

  #Create the model
  model = 

  #Start training with 500 epochs and 128 batch size

  #Save the model and all weights in 'fev-model' folder

train_fev_model()

### Test the Model

In [ ]:
def test_fev_model():
  #Load test data
  _, _, test_x, test_y = load_fev_data()

  #Load the model and all weights from 'fev-model' folder
  model = 
  
  #Run evaluation
  loss = 
  #Print the loss
  print(loss)

  #Run prediction
  predict_y = model.predict(test_x)

  #Print out the prediected value and actual value

  #Visualize prediction quality
  plot_prediction(predict_y, test_y)

test_fev_model()

### Solution

In [ ]:
def load_fev_data():
  #Load space separated data
  fev_data = pd.read_csv('lung-capacity.txt', delim_whitespace=True)

  #Split 90% of the data for training and 10% for test
  train = fev_data.sample(frac=0.9, random_state=200)
  test = fev_data.drop(train.index)

  #Extract the FEV column
  train_y = train["FEV"]
  test_y = test["FEV"]

  #Drop the FEV column
  train_x = train.drop("FEV", axis=1)
  test_x = test.drop("FEV", axis=1)

  #Return data
  return train_x, train_y, test_x, test_y

def create_fev_model(num_features):
  model = keras.Sequential()
  model.add(keras.layers.Dense(25, input_dim=num_features, activation='relu'))
  model.add(keras.layers.Dense(10, activation='relu'))
  model.add(keras.layers.Dense(5, activation='relu'))
  model.add(keras.layers.Dense(1)) #Readout

  model.compile(loss='mean_squared_error',
                  optimizer="adam")
  
  return model

def train_fev_model():
  #Start training
  train_x, train_y, _, _ = load_fev_data()
  model = create_fev_model(len(train_x.columns))
  model.fit(train_x, train_y, epochs=500, batch_size=128, verbose=True)

  #Save the model and all weights
  model.save('fev-model')

train_fev_model()

def test_fev_model():
  _, _, test_x, test_y = load_fev_data()

  #Load the model and all weights
  model = keras.models.load_model('fev-model')
  
  #Run evaluation
  result = model.evaluate(x=test_x, y=test_y)
  #Print the loss
  print(result)

  predict_y = model.predict(test_x)
  for y, y_ in zip(test_y.values, predict_y):
    # Print the actual price from log values
    print(y, y_)

  plot_prediction(predict_y, test_y)

test_fev_model()